I. Set Up

In [1]:
# PYTHON Imports 
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import ipywidgets as widgets
from IPython.display import clear_output
import statistics
# ASTROPHY Imports
import astropy 
from astropy.table import Table
from astropy.io import fits
# CIAO Imports
import ciao_contrib.runtool
from ciao_contrib.runtool import *
# CUSTOM Imports
from data_exploration_functions import *
from data_representation_functions import *

# Specify global path
global_path = '/Users/steven/Library/Mobile Documents/com~apple~CloudDocs/0-CfA/4-Data/Datasets'
global_folders = list_folders_fun(global_path)

# Select dataset
set_widget = widgets.Dropdown(options=global_folders[:],value=global_folders[0],description='Set :',disabled=False); set_widget

Dropdown(description='Set :', options=('All', 'Bona'), value='All')

Filter for N > 40 length

In [2]:
# Set ID
set_id = set_widget.value
# Reduce data for useful input to data representation
df_eventfiles = pd.read_csv(f'{global_path}/{set_id}/eventfiles-{set_id}.csv')
df_properties = pd.read_csv(f'{global_path}/{set_id}/properties-{set_id}.csv')


df_eventfiles_input, df_properties_input = data_reduction_fun(df_eventfiles, df_properties,global_path,set_id, min_counts = 50,unique_ids=True)

# df_eventfiles.to_csv(f'{global_path}/{set_id}/eventfiles-{set_id}.csv',index=False)
# df_properties.to_csv(f'{global_path}/{set_id}/properties-{set_id}.csv',index=False)

# # filter rows where there are less than 30 identical values in column 'B'
# mask = df['B'].isin(counts[counts >= 30].index)

# # create new DataFrame without the filtered rows
# new_df = df[mask]



In [6]:
print(len(df_eventfiles_input['obsreg_id'].unique()))
print(len(df_properties_input['obsreg_id'].unique()))
print(len(df_properties_input['obsreg_id']))

# count unique values in the 'obsreg_id' column
print(len(df_properties_input['obsreg_id'].unique())) # 11

# count total values in the 'obsreg_id' column
print(len(df_properties_input['obsreg_id'])) # 12

# identify duplicate values in the 'obsreg_id' column
duplicates = df_properties_input[df_properties_input.duplicated(['obsreg_id'], keep=False)]
print(duplicates)

62989
62989
63020
62989
63020
       cnts_aper_b  cnts_aperbkg_b  src_cnts_aper_b   flux_aper_b   hard_hm  \
95356          160              66       165.904303  1.252943e-13  0.914428   
95360           86              21        90.469791  1.252943e-13  0.963148   
96423          253             511       240.714066  2.899196e-14  0.234853   
96436           39             144        34.576335  2.899196e-14  0.224859   
95730           54              49        55.741454  2.057922e-15  0.999375   
...            ...             ...              ...           ...       ...   
85565           94              77        94.278217  1.181298e-13  0.357277   
17704          265            1179       217.610196  5.315461e-14  0.273579   
17703          203            1009       161.987217  5.315461e-14  0.428482   
17684           93              80        92.307372  4.227679e-14  0.833229   
17685          119              99       118.200006  4.227679e-14  0.809494   

        hard_hs   har

Filter Pileup Properties file

In [5]:
# Set ID
set_id = set_widget.value
# Reduce data for useful input to data representation
df_eventfiles = pd.read_csv(f'{global_path}/{set_id}/eventfiles-{set_id}.csv')
df_properties_full = pd.read_csv(f'{global_path}/{set_id}/properties-{set_id}-full.csv')
df_properties_full['obsreg_id'] = df_properties_full['obsid'].astype(str) + '_' + df_properties_full['region_id'].astype(str)
df_properties_pileup = pd.read_csv(f'{global_path}/{set_id}/properties-{set_id}-pileup.csv')
df_properties_pileup['obsreg_id'] = df_properties_pileup['obsid'].astype(str) + '_' + df_properties_pileup['region_id'].astype(str)
df_properties = df_properties_full[~df_properties_full['obsreg_id'].isin(df_properties_pileup['obsreg_id'].unique())]

df_properties.to_csv(f'{global_path}/{set_id}/properties-{set_id}.csv',index=False)



In [2]:
from astropy.coordinates import SkyCoord
import astropy.units as u

# Define the sky position in equatorial coordinates (RA and Dec in degrees)
ra = 0.032141223071846525 # 10.68458
dec = 50.10557203183708 # 41.26917

# Convert the sky position to the ICRS format using astropy
icrs_coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')

# Print the ICRS coordinates in decimal degrees
print(icrs_coord.ra.deg, icrs_coord.dec.deg)

0.032141223071846525 50.10557203183708
